<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatting with TeachableAgent

Conversational assistants based on LLMs can remember the current chat with the user, and can even demonstrate in-context learning of things that the user teaches the assistant during the chat. But these memories and learnings are lost once the chat is over, or when a single chat grows too long for the LLM to handle effectively. In subsequent chats, the user is forced to repeat any necessary instructions over and over.

`TeachableAgent` addresses these limitations by persisting user teachings across chat boundaries in long-term memory (a vector database). Memory is saved to disk at the end of each chat, then loaded from disk at the start of the next. Instead of copying all of memory into the context window, which would eat up valuable space, individual memories (called memos) are retrieved into context as needed. This allows the user to teach frequently used facts and skills to the agent just once, and have the agent remember them in later chats.

In making decisions about memo storage and retrieval, `TeachableAgent` calls an instance of `TextAnalyzerAgent` to analyze pieces of text in several different ways. This adds extra LLM calls involving a relatively small number of tokens. These calls can add a few seconds to the time a user waits for a response.

This notebook demonstrates how `TeachableAgent` can learn facts, preferences, and skills from users. To chat with `TeachableAgent` yourself, run [chat_with_teachable_agent.py](../test/agentchat/chat_with_teachable_agent.py).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [teachable] option.
```bash
pip install "pyautogen[teachable]"
```

In [1]:
%%capture --no-stderr
# %pip install "pyautogen[teachable]

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [2]:
import autogen

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location="",
    filter_dict={
        "model": ["gpt-4-32k"],
    },
)

print(config_list[0]["model"])

gpt-4-32k


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). After application of this particular filter, only the gpt-4 models are kept.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways if you prefer, e.g., loading from a YAML file.

## Construct Agents
For this walkthrough, we start by resetting the agent's memory store. This deletes any memories from prior conversations that may be stored on disk.

In [3]:
from autogen.agentchat.contrib.teachable_agent import TeachableAgent
from autogen import UserProxyAgent, AssistantAgent

llm_config = {
    "request_timeout": 1000,
    "config_list": config_list,
}

teach_config={
    "verbosity": 1,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    "reset_db": True,  # Set to True to start over with an empty database.
    "path_to_db_dir": "./tmp/notebook/teachable_agent_db",  # Path to the directory where the database will be stored.
    "recall_threshold": 1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
}

try:
    from termcolor import colored
except ImportError:
    def colored(x, *args, **kwargs):
        return x
    
autogen.ChatCompletion.start_logging()

agent = TeachableAgent(
    name="agent",
    system_message=AssistantAgent.DEFAULT_SYSTEM_MESSAGE,
    llm_config=llm_config,
    teach_config=teach_config)

user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "./coding",
        "use_docker": True,
    }
        
)




CLEARING MEMORY


## Learning new facts
Let's teach the agent some facts it doesn't already know, since they are more recent than GPT-4's training data.

In [4]:
text = "Find title and abstracts of 10 arxiv papers that show how are people studying trust calibration in AI based systems"
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Find title and abstracts of 10 arxiv papers that show how are people studying trust calibration in AI based systems

--------------------------------------------------------------------------------
agent (to user):

To achieve this, we will search the arXiv API for papers related to "trust calibration in AI-based systems" and extract the titles and abstracts from the search results. Execute the following Python code to perform the search and print the titles and abstracts of the top 10 results:

```python
import requests

def search_arxiv(query, num_results=10):
    url = "http://export.arxiv.org/api/query"
    payload = {
        "search_query": query,
        "start": 0,
        "max_results": num_results,
    }
    response = requests.get(url, params=payload)
    if response.status_code == 200:
        return response.text
    else:
        return None

def print_titles_and_abstracts(xml_text):
    import xml.etree.ElementTree as ET

    root = ET.fromstring(xml_te

In [5]:
text = "Summarize each paper. I like summaries to contain one sentence for title, innovation, and main result."
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

Summarize each paper. I like summaries to contain one sentence for title, innovation, and main result.

--------------------------------------------------------------------------------
agent (to user):

1. Title: Who Should I Trust: AI or Myself? Leveraging Human and AI Correctness Likelihood to Promote Appropriate Trust in AI-Assisted Decision-Making
Innovation: Models human and AI correctness likelihood to calibrate users' trust in decision-making.
Main result: Strategies using both human and AI correctness likelihood outperformed those relying on AI confidence alone, promoting more appropriate trust.

2. Title: Effect of Confidence and Explanation on Accuracy and Trust Calibration in AI-Assisted Decision Making
Innovation: Studies the effect of confidence score and local explanation on trust calibration in AI-assisted decision-making.
Main result: Confidence scores help calibrate trust, but trust calibration alone doesn't guarantee improved AI-assisted decision-mak

In [6]:
agent.learn_from_user_feedback()


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER

INPUT-OUTPUT PAIR ADDED TO VECTOR DATABASE:
  INPUT
    The task described is to create concise summaries of papers, highlighting their main elements.
  OUTPUT
    For a similar but different task, use the advice: "Summaries should contain one sentence for title, innovation, and main result."


# New Chat Session
Let's end our first chat here. The following function needs to be called at the end of each chat, so that `TeachableAgent` can store what the user has taught it.

In [7]:
text = "Find title and abstracts of 10 arxiv papers on explainable AI"
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Find title and abstracts of 10 arxiv papers on explainable AI

--------------------------------------------------------------------------------

agent (to user):

First, let me search arxiv for recent papers on explainable AI, then I will summarize them as you advised.

```python
import requests
from bs4 import BeautifulSoup

arxiv_url = "https://arxiv.org/search/?query=explainable+AI&start=0&searchtype=all"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

response = requests.get(arxiv_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
titles = soup.find_all('p', class_='title is-5 mathjax')
abstracts = soup.find_all('span', class_='abstract-full has-text-grey-dark mathjax')

for i in range(10):
    print(f"Title {i+1}: {titles[i].text.strip()}")
    print(f"Abstract {i+1}: {abstracts[i].text.strip()}\n")
```
Please run the code above to find 1

Now let's start a new chat by clearing the previous chat's history. At this point, common LLM-based assistants would forget everything from the last chat. But `TeachableAgent` can retrieve memories from its vector DB as needed, allowing it to recall and reason over facts that the user taught it in earlier conversations.

In [8]:
text = "Summarize each paper."
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

Summarize each paper.

--------------------------------------------------------------------------------

MEMOS APPENDED TO LAST USER MESSAGE...

# Memories that might help
- For a similar but different task, use the advice: "Summaries should contain one sentence for title, innovation, and main result."


agent (to user):

Here are the summaries for the papers provided, with one sentence for the title, innovation, and main result.

1. Title: Who Should I Trust: AI or Myself? Leveraging Human and AI Correctness
Innovation: The paper investigates optimal reliance on AI systems by combining human and AI decision-making.
Result: The authors find that trust calibration plays a significant role in leveraging the strengths of both human and AI decision-making.

2. Title: Effect of Confidence and Explanation on Accuracy and Trust Calibration in AI-Assisted Decision Making
Innovation: The study examines the impact of AI-generated confidence and explanation on decision-making in